<center>
    <img src="http://sct.inf.utfsm.cl/wp-content/uploads/2020/04/logo_di.png" style="width:60%">
    <h1> INF285 - Computación Científica </h1>
    <h2> Laboratorio 4</h2>
    <h2> [S]cientific [C]omputing [T]eam </a> </h2>
    <h2> 2025-1</h2>
</center>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import qr
from scipy.linalg import solve_triangular
from ipywidgets import interact, IntSlider

# **Contexto: Separando Nubes de Puntos**




Como vimos en el certamen 2 y en el contexto, podemos usar mínimos cuadrados para calcular una recta entre dos nubes de puntos a través de una tercera dimensión. **¿Qué pasaría si ahora hablamos de dos objetos en 3D?** Es decir, ahora nuestra nube de puntos que se describe como:

$\bigg(x_i^{(k)},y_i^{(k)},z_i^{(k)} \bigg)$ para $i \in \{1,...,n_k\}$

La respuesta es simple, tenemos que aplicar el mismo procedimiento y aumentar una dimensión, es decir:

$\bigg(x_i^{(1)},y_i^{(1)},z_i^{(1)}, 1 \bigg)$ para $i \in \{1,...,n_1\}$

$\bigg(x_i^{(2)},y_i^{(2)},z_i^{(2)}, -1 \bigg)$ para $i \in \{1,...,n_2\}$

Donde ahora nuestra recta separadora se convertiría en el plano $z = \hat{a}x + \hat{b}y + \hat{d}$ que separa las nubes de puntos, por lo tanto que buscamos el hiperplano $k = ax + by + cz + d$  **que se encuentre más cerca de cada punto** minimizando el error con la siguiente función:

$$
E_2(a,b,c,d) = \sum_{i=1}^{n_1} \bigg(1 - (ax_i^{(1)} + b y_i^{(1)} + c z_i^{(1)} + d) \bigg)^2 + \sum_{j=1}^{n_2} \bigg( -1 - (ax_j^{(2)} + b y_j^{(2)} + c z_j^{(2)} + d) \bigg)^2$$


Nuevamente para recuperar nuestra dimensión original igualaremos el hiperplano $k=0$

$$0 = ax + by + cz + d \implies cz = -ax -by -d \implies z = -\frac{a}{c}x -\frac{b}{c}y - \frac{d}{c}$$

Eso más nuestro parámetro $\frac{n_1}{n_2}$ nos permitirá conseguir el plano entre las nubes de puntos.

$$E_2(a,b,c,d) = \sum_{i=1}^{n_1} \bigg(1 - (ax_i^{(1)} + b y_i^{(1)} + c z_i^{(1)} + d) \bigg)^2 + \frac{n1}{n2} \sum_{j=1}^{n_2} \bigg( -1 - (ax_j^{(2)} + b y_j^{(2)} + c z_j^{(2)} + d) \bigg)^2$$

---

## Funciones disponibles

- **np.arange(n)**: Para n un número entero positivo entrega un vector de largo n con números enteros desde 0 hasta n-1
- **np.dot(a, b)**: Obtiene el producto interno entre el vector a y b. En caso de que a sea una matriz, entrega el producto matriz-vector respectivo. Para esto es posible usar el operador @.
- **np.{zeros, ones}(n)**: Genera un vector {nulo, de unos} de dimensión $n$.
- **np.{zeros, ones}((n,m))**: Genera una matriz {nula, de unos} de dimensión $n \times m$.
- **np.eyes(k)**: Genera la matriz identidad $I_k$ de dimensión $k \times k$
- **np.transpose(A)**: Obtiene la transpuesta de A, equivalente a A.T
- **np.sqrt(x)**: Calcula la raíz cuadrada elemento a elemento del vector x.
- **np.power(x, p)**: Calcula la potencia a la $p$ elemento a elemento del vector x.
- **np.any(x)**: Retorna Trues si algún elemento del array x tiene valor booleano True
- **np.all(x)**: Retorna Trues si todos los elementos del array x tienen valor booleano True
- **qr(A, mode="reduced")**: Factorización reducida QR de A.
- **solve_triangular(A, b, lower={False, True})**: Resuelve Ax = b, si A es triangular {superior, inferior}

---

# **Laboratorio**

In [2]:
def generate_cloud_3d(n, radius, center):
    """
    Genera n puntos aleatorios en una nube 3D dentro de una esfera de radio dado.

    Args:
        n (int): Número de puntos a generar.
        radius (float): Radio de la esfera en la que se generarán los puntos.
        center (tuple): Coordenadas del centro de la esfera (x, y, z).
    Returns:
        Tres arrays de numpy que representan las coordenadas x, y, z de los puntos generados.
    """
    # Creating Random Number Generator (rng) with seed=99.
    rng = np.random.default_rng(seed=99)
    r = radius * np.cbrt(rng.random(n))  # distribución uniforme en el volumen
    theta = rng.uniform(0, 2 * np.pi, n)  # ángulo azimutal
    phi = np.arccos(1 - 2 * rng.random(n))  # ángulo polar

    # Conversión a coordenadas cartesianas
    x = center[0] + r * np.sin(phi) * np.cos(theta)
    y = center[1] + r * np.sin(phi) * np.sin(theta)
    z = center[2] + r * np.cos(phi)

    return x, y, z

Concretamente vamos a intentar encontrar el plano en común que separe $k=2$ nubes de puntos, con la siguiente estructura:
$$
\{x^{(k)}_i, y^{(k)}_i, z^{(k)}_i \}
$$

In [3]:
#--- NO MODIFICAR ESTA CELDA ---- | --- NO MODIFICAR ESTA CELDA ---- | --- NO MODIFICAR ESTA CELDA ---- |
n1,n2,n3 = 100,20,40
x1, y1, z1 = generate_cloud_3d(n1, 5.0, (10, 10, 10))
x2, y2, z2 = generate_cloud_3d(n2, 2.0, (1, 1, 1))

def plot_view(angle=0):
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x1, y1, z1, s=1, alpha=0.5, c='blue', label='Cloud 1')
    ax.scatter(x2, y2, z2, s=1, alpha=0.5, c='red', label='Cloud 2')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.legend()
    ax.view_init(elev=20, azim=angle)
    ax.set_title(f'angle: {angle}°')
    plt.tight_layout()
    plt.show()

interact(plot_view, angle=IntSlider(min=0, max=360, step=5, value=0))
#--- NO MODIFICAR ESTA CELDA ---- | --- NO MODIFICAR ESTA CELDA ---- | --- NO MODIFICAR ESTA CELDA ---- |

interactive(children=(IntSlider(value=0, description='angle', max=360, step=5), Output()), _dom_classes=('widg…

<function __main__.plot_view(angle=0)>

---

# Pregunta 1 (10 Pts)

¿Cúal es el proposito del párametro $\frac{n_1}{n_2}$?

---

**Respuesta:** El parámetro$\frac{n_1}{n_2}$ se utiliza para ponderar la contribución de la segunda nube de puntos en el cálculo del plano separador utilizando mínimos cuadrados. Su propósito es **corregir el sesgo que ocurre cuando hay un desequilibrio en la cantidad de puntos entre las dos nubes**.

Sin este parámetro, la nube con más puntos tendría mayor influencia sobre el plano, haciendo que éste se ajuste más a ella y pueda no quedar correctamente entre ambas. Al aplicar este factor de ponderación, se garantiza que ambas nubes tengan la misma importancia relativa en el error total, sin importar su tamaño.

Esto permite una estimación del plano más justa, lo que es crucial para determinar correctamente si existe o no una posible colisión entre los objetos representados por las nubes de puntos.

---


# Pregunta 2 (20 pts)

Desarrolle la función, `build_A_and_b` la cual tiene que retornar las matrices `A,b` asociadas al problema de mínimos cuadrados $Ax=b$ que se busca resolver.

In [5]:
def build_A_and_b(x1, y1, z1, x2, y2, z2, n1, n2):

    """
    Genera la matriz A asociada al problema de minimos cuadrados, a partir de dos nubes de puntos 3D.

    Args:
        x1, y1, z1 (array [n1 x 1]): Coordenadas de la primera nube de puntos..
        x2, y2, z2 (array [n2 x 1]): Coordenadas de la segunda nube de puntos.
        n1 (int): Número de puntos en la primera nube.
        n2 (int): Número de puntos en la segunda nube.
    Returns:
        A,b
        A: Matriz numpy que representa la matriz A [(n1+n2) x 4] del problema de mínimos cuadrados.
        b: Matriz numpy que representa el vector B [(n1+n2) x 1] del problema de mínimos cuadrados.

    """
    # acá va su codigo
    #--------------------------------
    A = np.zeros((n1 + n2, 4))
    b = np.zeros(n1 + n2)

    A[0:n1, 0] = x1
    A[0:n1, 1] = y1
    A[0:n1, 2] = z1
    A[0:n1, 3] = 1.0
    b[0:n1] = 1.0


    peso = np.sqrt(n1 / n2)

    A[n1:n1+n2, 0] = x2 * peso
    A[n1:n1+n2, 1] = y2 * peso
    A[n1:n1+n2, 2] = z2 * peso
    A[n1:n1+n2, 3] = 1.0 * peso
    b[n1:n1+n2] = -1.0 * peso
    #--------------------------------
    return A,b



Matriz A:
[[ 8.65136117e+00  1.37126861e+01  1.05220455e+01  1.00000000e+00]
 [ 7.46548306e+00  6.73443083e+00  9.99145636e+00  1.00000000e+00]
 [ 9.90058020e+00  1.39607629e+01  9.45161882e+00  1.00000000e+00]
 [ 1.42492936e+01  1.25142517e+01  9.60464448e+00  1.00000000e+00]
 [ 6.67871762e+00  9.78044077e+00  7.35450731e+00  1.00000000e+00]
 [ 9.26934527e+00  1.38485569e+01  1.13441837e+01  1.00000000e+00]
 [ 7.63550915e+00  1.00402961e+01  7.70225310e+00  1.00000000e+00]
 [ 8.31804634e+00  1.24482891e+01  1.28373831e+01  1.00000000e+00]
 [ 1.03303833e+01  6.53196874e+00  8.29030653e+00  1.00000000e+00]
 [ 9.79911285e+00  1.37678784e+01  8.06469559e+00  1.00000000e+00]
 [ 9.42370448e+00  5.24134815e+00  9.07807182e+00  1.00000000e+00]
 [ 1.20598123e+01  1.18154987e+01  1.15077249e+01  1.00000000e+00]
 [ 1.06278955e+01  6.75921489e+00  9.26492237e+00  1.00000000e+00]
 [ 1.29906617e+01  9.94761840e+00  1.21030334e+01  1.00000000e+00]
 [ 9.72256618e+00  1.13835712e+01  1.29084948e+01  1

---

# Pregunta 3 (20 Pts)

Construya la función `common_plane` la cual retorna los coeficientes $A,B,C$ y $D$ que definen el plano común $Ax + By + Cz + D = 0$ entre las nubes de puntos.

In [6]:
def common_plane(x1, y1, z1, x2, y2, z2, n1, n2):
    """
    Calcula los coeficientes del plano común a dos nubes de puntos 3D utilizando mínimos cuadrados.
    Args:
        x1, y1, z1 (array [n1 x 1]): Coordenadas de la primera nube de puntos.
        x2, y2, z2 (array [n2 x 1]): Coordenadas de la segunda nube de puntos.
        n1 (int): Número de puntos en la primera nube.
        n2 (int): Número de puntos en la segunda nube.
    Returns:
        array [4 x 1]: Coeficientes del plano común (A, B, C, D) de la forma Ax + By + Cz + D = 0.
    """
    # acá va su codigo
    #--------------------------------
    A, b = build_A_and_b(x1, y1, z1, x2, y2, z2, n1, n2)
    Q,R= qr(A, mode="reduced")
    Qt_b = np.dot(np.transpose(Q), b)
    coef= solve_triangular(R, Qt_b)


    #--------------------------------
    return coef


# **Visualizar el Plano**

In [7]:
#--- NO MODIFICAR ESTA CELDA ---- | --- NO MODIFICAR ESTA CELDA ---- | --- NO MODIFICAR ESTA CELDA ---- |
n1,n2 = 200,10
x1, y1, z1 = generate_cloud_3d(n1, 5.0, (10, 10, 10))
x2, y2, z2 = generate_cloud_3d(n2, 2.0, (1, 1, 1))

A,B,C,D = common_plane(x1, y1, z1, x2, y2, z2, n1, n2)

def plot_view(angle=0):
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')

    ax.scatter(x1, y1, z1, s=1, alpha=0.5, c='blue', label='Cloud 1')
    ax.scatter(x2, y2, z2, s=1, alpha=0.5, c='red', label='Cloud 2')

    all_x = np.concatenate([x1, x2])
    all_y = np.concatenate([y1, y2])

    x_min, x_max = all_x.min(), all_x.max()
    y_min, y_max = all_y.min(), all_y.max()

    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 20),
                         np.linspace(y_min, y_max, 20))

    # Calcular Z usando la ecuación del plano: Ax + By + Cz + D = 0
    # Despejando z: z = -(Ax + By + D) / C
    if abs(C) > 1e-8:
        zz = -(A * xx + B * yy + D) / C
        ax.plot_surface(xx, yy, zz, alpha=0.3, color='green', label='Plano común')
    else:
        print("El plano es vertical (C ≈ 0), no se puede graficar como superficie z=f(x,y)")

    ax.legend()
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.view_init(elev=20, azim=angle)
    ax.set_title(f'Vista azimutal: {angle}°')
    plt.tight_layout()
    plt.show()

interact(plot_view, angle=IntSlider(min=0, max=360, step=5, value=0))
#--- NO MODIFICAR ESTA CELDA ---- | --- NO MODIFICAR ESTA CELDA ---- | --- NO MODIFICAR ESTA CELDA ---- |

interactive(children=(IntSlider(value=0, description='angle', max=360, step=5), Output()), _dom_classes=('widg…

<function __main__.plot_view(angle=0)>

# Pregunta 4 (30 Pts)

En el gráfico anterior, deberíamos poder ver cómo el plano que generamos sí logra separar ambos cúmulos de puntos, pero aún no hemos desarrollado una función para determinar si hay o no colisiones. Desarrolle la función `check_collision`, que retorne `True` si hay un punto que este sobre o del lado incorrecto del plano y `False` en caso contrario.

In [8]:
def check_collision(x1, y1, z1, x2, y2, z2, A,B, C, D):
    """
    Calcula los coeficientes del plano común a dos nubes de puntos 3D utilizando mínimos cuadrados.
    Args:
        x1, y1, z1 (array [n1 x 1]): Coordenadas de la primera nube de puntos.
        x2, y2, z2 (array [n2 x 1]): Coordenadas de la segunda nube de puntos.
        A, B, C, D (float): Coeficientes del plano común de la forma Ax + By + Cz + D = 0.
    Returns:
        bool: True si hay colisión, False en caso contrario.
    """
    # acá va su codigo
    #--------------------------------
    eval1= A*x1+B*y1+C*z1+D
    test1=np.any(eval1<0)
    eval2= A*x2+B*y2+C*z2+D
    test2=np.any(eval2>0)
    collision=test1 or test2

    #--------------------------------
    return collision



Si el código funciona correctamente la siguiente celda debería arrojar `False`

In [9]:
#--- NO MODIFICAR ESTA CELDA ---- | --- NO MODIFICAR ESTA CELDA ---- | --- NO MODIFICAR ESTA CELDA ---- |
check_collision(x1, y1, z1, x2, y2, z2, A, B, C, D)
#--- NO MODIFICAR ESTA CELDA ---- | --- NO MODIFICAR ESTA CELDA ---- | --- NO MODIFICAR ESTA CELDA ---- |

np.False_

Ahora podemos agregar un punto que genere una colisión y ver si la función retorna `True`.

In [10]:
#--- NO MODIFICAR ESTA CELDA ---- | --- NO MODIFICAR ESTA CELDA ---- | --- NO MODIFICAR ESTA CELDA ---- |
x_new1 = np.concatenate([x1, [1]])
y_new1 = np.concatenate([y1, [1]])
z_new1 = np.concatenate([z1, [1]])
check_collision(x_new1, y_new1, z_new1, x2, y2, z2, A, B, C, D)
#--- NO MODIFICAR ESTA CELDA ---- | --- NO MODIFICAR ESTA CELDA ---- | --- NO MODIFICAR ESTA CELDA ---- |

np.True_

# Pregunta 5 (20 Pts)

¿Cómo se puede generalizar el problema de encontrar el plano o hiperplano común entre dos nubes de puntos si seguimos aumentando las dimensiones? Describa el proceso a seguir en el caso de nubes de puntos pertenecientes a $\mathbb{R}^n$, indicando las dimensiones de cada matriz involucrada.

---

**Respuesta:** Para el caso de nubes que pertenezcan a  $\mathbb{R}^n$ se debe aumentar la dimensión del problema, es decir generar los puntos que pertenezcan a $\mathbb{R}^{n+1}$, para de esa forma poder construir el hiperplano común, que pasa por ambas nubes. Este se representa de la siguiente forma:
$$
f(\mathbf{x}) = w_1 x_1 + w_2 x_2 + \dots + w_n x_n + b = 0
$$
Este hiperplano separador se representa como un vector de parámetros:
$$
\boldsymbol{\beta} = [w_1, w_2, \dots, w_n, b]^\top \in \mathbb{R}^{n+1}
$$
Esto nos permite generar el problema de mínimos cuadrados que hay que resolver, donde se debe encontrar los coeficientes que minimizen el plano. Para ello debemos construir el siguiente sistema:
- La matriz de diseño$$
A \in \mathbb{R}^{(n_1 + n_2) \times (n + 1)}
$$
se forma colocando en cada fila un punto extendido con un 1 al final:  
$$
[x_1, x_2, \dots, x_n, 1]
$$

- El **vector de etiquetas**
$$
b \in \mathbb{R}^{(n_1 + n_2)}
$$ contiene:
  - \(+1\) para puntos de la primera nube
  - \(-1\) para la segunda nube

- Con el factor $\frac{n_1}{n_2}$, se ponderan los puntos de la segunda nube con el factor:

$$
\sqrt{\frac{n_1}{n_2}}
$$
para todas las filas que contengan los puntos de la segunda nube
Luego se resuelve el problema de mínimos cuadrados mediante la factorización QR
$$
  A = QR \Rightarrow R x = Q^\top b
$$

y despues se usa la función check_collision para ver si existe alguna colisión, adaptandola a la dimensiones del problema en $\mathbb{R}^{n}$
---